In [1]:
import pandas as pd
import numpy as np
from gurobipy import GRB
import gurobipy as gb

In [2]:
# (a)

In [3]:
# Create the optimization model
model = gb.Model('Can2Oil_Transportation_Optimization')

Set parameter Username
Academic license - for non-commercial use only - expires 2025-01-15


In [4]:
# Decision variables
## Let x[i, j] denote the amount of canola oil transported directly from production facility i to refinement center j.
## Let y[i, k] denote the amount of canola oil transported from production facility i to transshipment hub k.
## Let z[k, j] denote the amount of canola oil transported from transshipment hub k to refinement center j.

x = model.addVars(25, 5, name="Direct Production Facilities")
y = model.addVars(15, 2, name="Transshipment Production Facilities")
z = model.addVars(2, 5, name="Transshipment Hubs")

In [5]:
direct_prod_capacity_df = pd.read_csv('/Users/shiyini/Desktop/YorkU Schulich💚/Term 3 - Winter/OMIS 6000 F/Assignment/Capacity_for_Direct_Production_Facilities.csv')
trans_prod_capacity_df = pd.read_csv('/Users/shiyini/Desktop/YorkU Schulich💚/Term 3 - Winter/OMIS 6000 F/Assignment/Capacity_for_Transship_Production_Facilities.csv')
trans_capacity_df = pd.read_csv('/Users/shiyini/Desktop/YorkU Schulich💚/Term 3 - Winter/OMIS 6000 F/Assignment/Capacity_for_Transship_Distribution_Centers.csv')
refinement_demand_df = pd.read_csv('/Users/shiyini/Desktop/YorkU Schulich💚/Term 3 - Winter/OMIS 6000 F/Assignment/Refinement_Demand.csv')

direct_prod_refinement_cost_df = pd.read_csv('/Users/shiyini/Desktop/YorkU Schulich💚/Term 3 - Winter/OMIS 6000 F/Assignment/Cost_Production_to_Refinement.csv')
trans_prod_trans_cost_df = pd.read_csv('/Users/shiyini/Desktop/YorkU Schulich💚/Term 3 - Winter/OMIS 6000 F/Assignment/Cost_Production_to_Transshipment.csv')
trans_refinement_cost_df = pd.read_csv('/Users/shiyini/Desktop/YorkU Schulich💚/Term 3 - Winter/OMIS 6000 F/Assignment/Cost_Transshipment_to_Refinement.csv')

In [6]:
direct_prod_capacity = direct_prod_capacity_df['Capacity']
trans_prod_capacity = trans_prod_capacity_df['Capacity']
trans_capacity = trans_capacity_df['Capacity']
refinement_demand = refinement_demand_df['Demand']

direct_prod_refinement_cost = [group["Cost"].tolist() for _, group in direct_prod_refinement_cost_df.groupby("ProductionFacility")]
trans_prod_trans_cost = [group["Cost"].tolist() for _, group in trans_prod_trans_cost_df.groupby("ProductionFacility")]
trans_refinement_cost = [group["Cost"].tolist() for _, group in trans_refinement_cost_df.groupby("TransshipmentHub")]

In [7]:
# Constraints

## Capacity constraints for direct production facility
for i in range(25):
    model.addConstr(gb.quicksum(x[i, j] for j in range(5)) <= direct_prod_capacity[i], name=f"Capacity_Constraint_Direct_Production_{i}")

## Capacity constraints for transshipment production facility
for i in range(15):
    model.addConstr(gb.quicksum(y[i, k] for k in range(2)) <= trans_prod_capacity[i], name=f"Capacity_Constraint_Transship_Production_{i}")

## Capacity constraints for transshipment hub
for k in range(2):
    # Flow conservation: Inflow equals outflow
    model.addConstr(gb.quicksum(y[i, k] for i in range(15)) == gb.quicksum(z[k, j] for j in range(5)), name=f"Flow_Conservation_Transship_Hub_{k}")
    
    # Capacity constraint: Inflow does not exceed hub capacity
    model.addConstr(gb.quicksum(y[i, k] for i in range(15)) <= trans_capacity[k], name=f"Capacity_Constraint_Transship_Hub_{k}")

## Demand constraints for refinement centers
for j in range(5):
    model.addConstr(gb.quicksum(x[i, j] for i in range(25)) + gb.quicksum(z[k, j] for k in range(2)) == refinement_demand[j], name=f"Demand_Constraint_Refinement_{j}")


In [8]:
# The objective function
total_cost = gb.quicksum(gb.quicksum(direct_prod_refinement_cost[i][j] * x[i, j] for j in range(5)) for i in range(25)) + \
             gb.quicksum(gb.quicksum(trans_prod_trans_cost[i][k] * y[i, k] for k in range(2)) for i in range(15)) + \
             gb.quicksum(gb.quicksum(trans_refinement_cost[k][j] * z[k, j] for j in range(5)) for k in range(2))

model.setObjective(total_cost, GRB.MINIMIZE)

In [9]:
# Optimally solve the problem
model.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 23.2.0 23C71)

CPU model: Apple M1 Pro
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 49 rows, 165 columns and 360 nonzeros
Model fingerprint: 0x9706ddb1
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [6e-01, 6e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+02, 2e+03]
Presolve time: 0.00s
Presolved: 49 rows, 165 columns, 360 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.7230583e+04   8.266000e+03   0.000000e+00      0s
      36    2.4188585e+04   0.000000e+00   0.000000e+00      0s

Solved in 36 iterations and 0.00 seconds (0.00 work units)
Optimal objective  2.418858517e+04


In [10]:
# Output the results
if model.status == GRB.Status.OPTIMAL:
    print('Optimal Total Transportation Cost:', model.objVal)

    for i, j in x.keys():
        if x[i, j].X > 0:
            print(f"From Production Facility {i} to Refinement Center {j}: {x[i, j].X} units")
    
    for i, k in y.keys():
        if y[i, k].X > 0:
            print(f"From Production Facility {i} to Transshipment Hub {k}: {y[i, k].X} units")

    for k, j in z.keys():
        if z[k, j].X > 0:
            print(f"From Transshipment Hub {k} to Refinement Center {j}: {z[k, j].X} units")

Optimal Total Transportation Cost: 24188.58516544768
From Production Facility 0 to Refinement Center 3: 462.0 units
From Production Facility 1 to Refinement Center 1: 103.0 units
From Production Facility 2 to Refinement Center 2: 460.0 units
From Production Facility 4 to Refinement Center 3: 86.0 units
From Production Facility 5 to Refinement Center 1: 217.0 units
From Production Facility 7 to Refinement Center 4: 521.0 units
From Production Facility 8 to Refinement Center 4: 548.0 units
From Production Facility 10 to Refinement Center 4: 354.0 units
From Production Facility 11 to Refinement Center 0: 7.0 units
From Production Facility 11 to Refinement Center 2: 404.0 units
From Production Facility 12 to Refinement Center 0: 104.0 units
From Production Facility 13 to Refinement Center 4: 155.0 units
From Production Facility 14 to Refinement Center 3: 285.0 units
From Production Facility 15 to Refinement Center 0: 109.0 units
From Production Facility 17 to Refinement Center 1: 351.0 uni

In [11]:
#(b)
# proportion of canola oil is transshipped

# Calculate the total transshipped amount
total_transshipped = sum(y[i, k].X for i in range(15) for k in range(2) if y[i, k].X > 0)

# Calculate the total produced amount
total_produced = sum(x[i, j].X for i in range(25) for j in range(5) if x[i, j].X > 0) + total_transshipped

# Calculate the proportion
proportion_transshipped = total_transshipped / total_produced

print(f"Proportion of Canola Oil Transshipped: {proportion_transshipped:.2%}")


Proportion of Canola Oil Transshipped: 31.74%


In [12]:
total_produced

8728.0

In [13]:
# (c)

In [14]:
model2 = model.copy()

In [15]:
# Define a penalty cost per unit of transshipped oil
transshipment_penalty_cost = 0.5

# Update the objective function to include the transshipment penalty
total_cost_with_penalty = (
    gb.quicksum(direct_prod_refinement_cost[i][j] * model2.getVarByName(f'Direct Production Facilities[{i},{j}]')
                for i in range(25) for j in range(5)) +
    gb.quicksum(trans_prod_trans_cost[i][k] * model2.getVarByName(f'Transshipment Production Facilities[{i},{k}]')
                for i in range(15) for k in range(2)) +
    gb.quicksum(trans_refinement_cost[k][j] * model2.getVarByName(f'Transshipment Hubs[{k},{j}]')
                for k in range(2) for j in range(5)) +
    transshipment_penalty_cost * gb.quicksum(model2.getVarByName(f'Transshipment Production Facilities[{i},{k}]')
                                            for i in range(15) for k in range(2))
)

model2.setObjective(total_cost_with_penalty, GRB.MINIMIZE)

In [16]:
model2.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 23.2.0 23C71)

CPU model: Apple M1 Pro
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 49 rows, 165 columns and 360 nonzeros
Model fingerprint: 0x855cb181
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 6e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+02, 2e+03]
Presolve time: 0.00s
Presolved: 49 rows, 165 columns, 360 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.7230583e+04   8.266000e+03   0.000000e+00      0s
      34    2.5553012e+04   0.000000e+00   0.000000e+00      0s

Solved in 34 iterations and 0.00 seconds (0.00 work units)
Optimal objective  2.555301173e+04


In [17]:
# Output the results
if model2.status == GRB.Status.OPTIMAL:
    print('Optimal Total Transportation Cost:', model2.objVal)

    for i in range(25):
        for j in range(5):
            variable_name = f"Direct Production Facilities[{i},{j}]"
            variable = model2.getVarByName(variable_name)
            if variable is not None and variable.X > 0:
                print(f"From Production Facility {i} to Refinement Center {j}: {variable.X} units")

    for i in range(15):
        for k in range(2):
            variable_name = f"Transshipment Production Facilities[{i},{k}]"
            variable = model2.getVarByName(variable_name)
            if variable is not None and variable.X > 0:
                print(f"From Production Facility {i} to Transshipment Hub {k}: {variable.X} units")

    for k in range(2):
        for j in range(5):
            variable_name = f"Transshipment Hubs[{k},{j}]"
            variable = model2.getVarByName(variable_name)
            if variable is not None and variable.X > 0:
                print(f"From Transshipment Hub {k} to Refinement Center {j}: {variable.X} units")

Optimal Total Transportation Cost: 25553.01172601651
From Production Facility 0 to Refinement Center 3: 462.0 units
From Production Facility 1 to Refinement Center 1: 103.0 units
From Production Facility 2 to Refinement Center 2: 460.0 units
From Production Facility 4 to Refinement Center 3: 86.0 units
From Production Facility 5 to Refinement Center 1: 217.0 units
From Production Facility 6 to Refinement Center 2: 145.0 units
From Production Facility 7 to Refinement Center 4: 521.0 units
From Production Facility 8 to Refinement Center 4: 548.0 units
From Production Facility 10 to Refinement Center 4: 361.0 units
From Production Facility 11 to Refinement Center 0: 159.0 units
From Production Facility 11 to Refinement Center 2: 252.0 units
From Production Facility 12 to Refinement Center 0: 104.0 units
From Production Facility 13 to Refinement Center 4: 155.0 units
From Production Facility 14 to Refinement Center 3: 285.0 units
From Production Facility 15 to Refinement Center 0: 109.0 un

In [18]:
# Calculate the total transshipped amount
total_transshipped2 = sum(model2.getVarByName(f'Transshipment Production Facilities[{i},{k}]').X for i in range(15) for k in range(2))

# Calculate the total produced amount
total_produced2 = sum(model2.getVarByName(f'Direct Production Facilities[{i},{j}]').X for i in range(25) for j in range(5)) + total_transshipped2

# Calculate the proportion
proportion_transshipped2 = total_transshipped2 / total_produced2

print(f"Proportion of Canola Oil Transshipped: {proportion_transshipped2:.2%}")

Proportion of Canola Oil Transshipped: 30.00%


In [19]:
# (d)

In [20]:
model3 = model.copy()

In [21]:
# Add a constraint for the total transshipment not exceeding 400
transshipment_limit = 2200
total_transshipment = gb.quicksum(model3.getVarByName(f"Transshipment Production Facilities[{i},{k}]") for i in range(15) for k in range(2))
model3.addConstr(total_transshipment <= transshipment_limit, name="Transshipment_Limit")

<gurobi.Constr *Awaiting Model Update*>

In [22]:
model3.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 23.2.0 23C71)

CPU model: Apple M1 Pro
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 50 rows, 165 columns and 390 nonzeros
Model fingerprint: 0x6258b619
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [6e-01, 6e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+02, 2e+03]
Presolve time: 0.00s
Presolved: 50 rows, 165 columns, 390 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.7230583e+04   8.266000e+03   0.000000e+00      0s
      37    2.4698685e+04   0.000000e+00   0.000000e+00      0s

Solved in 37 iterations and 0.00 seconds (0.00 work units)
Optimal objective  2.469868473e+04


In [23]:
# Output the results
if model3.status == GRB.Status.OPTIMAL:
    print('Optimal Total Transportation Cost:', model3.objVal)

    for i in range(25):
        for j in range(5):
            variable_name = f"Direct Production Facilities[{i},{j}]"
            variable = model3.getVarByName(variable_name)
            if variable is not None and variable.X > 0:
                print(f"From Production Facility {i} to Refinement Center {j}: {variable.X} units")

    for i in range(15):
        for k in range(2):
            variable_name = f"Transshipment Production Facilities[{i},{k}]"
            variable = model3.getVarByName(variable_name)
            if variable is not None and variable.X > 0:
                print(f"From Production Facility {i} to Transshipment Hub {k}: {variable.X} units")

    for k in range(2):
        for j in range(5):
            variable_name = f"Transshipment Hubs[{k},{j}]"
            variable = model3.getVarByName(variable_name)
            if variable is not None and variable.X > 0:
                print(f"From Transshipment Hub {k} to Refinement Center {j}: {variable.X} units")


Optimal Total Transportation Cost: 24698.684727093372
From Production Facility 0 to Refinement Center 3: 462.0 units
From Production Facility 1 to Refinement Center 1: 103.0 units
From Production Facility 2 to Refinement Center 2: 460.0 units
From Production Facility 3 to Refinement Center 0: 131.0 units
From Production Facility 4 to Refinement Center 2: 227.0 units
From Production Facility 5 to Refinement Center 1: 217.0 units
From Production Facility 6 to Refinement Center 2: 205.0 units
From Production Facility 7 to Refinement Center 4: 521.0 units
From Production Facility 8 to Refinement Center 4: 548.0 units
From Production Facility 10 to Refinement Center 4: 361.0 units
From Production Facility 11 to Refinement Center 0: 28.0 units
From Production Facility 11 to Refinement Center 2: 383.0 units
From Production Facility 12 to Refinement Center 0: 104.0 units
From Production Facility 13 to Refinement Center 4: 155.0 units
From Production Facility 14 to Refinement Center 3: 285.0 un

In [24]:
# Calculate the total transshipped amount
total_transshipped3 = sum(model3.getVarByName(f'Transshipment Production Facilities[{i},{k}]').X for i in range(15) for k in range(2))

# Calculate the total produced amount
total_produced3 = sum(model3.getVarByName(f'Direct Production Facilities[{i},{j}]').X for i in range(25) for j in range(5)) + total_transshipped3

# Calculate the proportion
proportion_transshipped3 = total_transshipped3 / total_produced3

print(f"Proportion of Canola Oil Transshipped: {proportion_transshipped3:.2%}")

Proportion of Canola Oil Transshipped: 25.21%


In [25]:
# (e)
model4 = gb.Model('Can2Oil_Re_Shoring')

In [27]:
x_4 = model4.addVars(25, 5, name="Direct Production Facilities")
y_4 = model4.addVars(15, 2, name="Transshipment Production Facilities")
z_4 = model4.addVars(2, 5, name="Transshipment Hubs")

In [28]:
# Constraints

## Capacity constraints for direct production facility
for i in range(25):
    model4.addConstr(gb.quicksum(x_4[i, j] for j in range(5)) <= direct_prod_capacity[i], name=f"Capacity_Constraint_Direct_Production_{i}")

## Capacity constraints for transshipment production facility
for i in range(15):
    model4.addConstr(gb.quicksum(y_4[i, k] for k in range(2)) <= trans_prod_capacity[i], name=f"Capacity_Constraint_Transship_Production_{i}")

## Capacity constraints for transshipment hub
for k in range(2):
    # Flow conservation: Inflow equals outflow
    model4.addConstr(gb.quicksum(y_4[i, k] for i in range(15)) == gb.quicksum(z_4[k, j] for j in range(5)), name=f"Flow_Conservation_Transship_Hub_{k}")
    
    # Capacity constraint: Inflow does not exceed hub capacity
    model4.addConstr(gb.quicksum(y_4[i, k] for i in range(15)) <= trans_capacity[k], name=f"Capacity_Constraint_Transship_Hub_{k}")

## Demand constraints for refinement centers
for j in range(5):
    model4.addConstr(gb.quicksum(x_4[i, j] for i in range(25)) + gb.quicksum(z_4[k, j] for k in range(2)) == refinement_demand[j], name=f"Demand_Constraint_Refinement_{j}")

In [38]:
direct_prod_refinement_cost_df

,ProductionFacility,RefinementCenter,Cost
0,1,1,4.252733
1,1,2,4.567726
2,1,3,4.696484
3,1,4,2.678741
4,1,5,4.272451
...,...,...,...
120,25,1,4.384176
121,25,2,5.943448
122,25,3,4.999220
123,25,4,4.154833


In [40]:
closer_producer_discount = 0.8  # 20% discount for closer producers

# Apply the discount to the 'Cost' column for the specified ProductionFacility range
direct_prod_refinement_cost_df['DiscountedCost'] = direct_prod_refinement_cost_df.apply(
    lambda row: row['Cost'] * closer_producer_discount if 1 <= row['ProductionFacility'] <= 15 else row['Cost'],
    axis=1
)
direct_prod_refinement_cost_df

,ProductionFacility,RefinementCenter,Cost,DiscountedCost
0,1,1,4.252733,3.402186
1,1,2,4.567726,3.654180
2,1,3,4.696484,3.757188
3,1,4,2.678741,2.142993
4,1,5,4.272451,3.417961
...,...,...,...,...
120,25,1,4.384176,4.384176
121,25,2,5.943448,5.943448
122,25,3,4.999220,4.999220
123,25,4,4.154833,4.154833


In [41]:
direct_prod_refinement_cost4 = [group["DiscountedCost"].tolist() for _, group in direct_prod_refinement_cost_df.groupby("ProductionFacility")]

In [43]:
# The objective function
total_cost4 = gb.quicksum(gb.quicksum(direct_prod_refinement_cost4[i][j] * x_4[i, j] for j in range(5)) for i in range(25)) + \
             gb.quicksum(gb.quicksum(trans_prod_trans_cost[i][k] * y_4[i, k] for k in range(2)) for i in range(15)) + \
             gb.quicksum(gb.quicksum(trans_refinement_cost[k][j] * z_4[k, j] for j in range(5)) for k in range(2))

model4.setObjective(total_cost4, GRB.MINIMIZE)

In [44]:
model4.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 23.2.0 23C71)

CPU model: Apple M1 Pro
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 49 rows, 165 columns and 360 nonzeros
Model fingerprint: 0xdd6d05c2
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [6e-01, 6e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+02, 2e+03]
Presolve time: 0.00s
Presolved: 49 rows, 165 columns, 360 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.5984599e+04   7.745000e+03   0.000000e+00      0s
      34    2.1821661e+04   0.000000e+00   0.000000e+00      0s

Solved in 34 iterations and 0.00 seconds (0.00 work units)
Optimal objective  2.182166116e+04


In [45]:
# Output the results
if model4.status == GRB.Status.OPTIMAL:
    print('Optimal Total Transportation Cost:', model4.objVal)

    for i, j in x_4.keys():
        if x_4[i, j].X > 0:
            print(f"From Production Facility {i} to Refinement Center {j}: {x_4[i, j].X} units")
    
    for i, k in y_4.keys():
        if y_4[i, k].X > 0:
            print(f"From Production Facility {i} to Transshipment Hub {k}: {y_4[i, k].X} units")

    for k, j in z_4.keys():
        if z_4[k, j].X > 0:
            print(f"From Transshipment Hub {k} to Refinement Center {j}: {z_4[k, j].X} units")

Optimal Total Transportation Cost: 21821.661162218505
From Production Facility 0 to Refinement Center 3: 462.0 units
From Production Facility 1 to Refinement Center 1: 103.0 units
From Production Facility 2 to Refinement Center 2: 460.0 units
From Production Facility 3 to Refinement Center 0: 40.0 units
From Production Facility 4 to Refinement Center 2: 141.0 units
From Production Facility 4 to Refinement Center 3: 86.0 units
From Production Facility 5 to Refinement Center 1: 217.0 units
From Production Facility 6 to Refinement Center 2: 205.0 units
From Production Facility 7 to Refinement Center 4: 521.0 units
From Production Facility 8 to Refinement Center 4: 548.0 units
From Production Facility 10 to Refinement Center 4: 361.0 units
From Production Facility 11 to Refinement Center 0: 228.0 units
From Production Facility 11 to Refinement Center 2: 183.0 units
From Production Facility 12 to Refinement Center 0: 104.0 units
From Production Facility 13 to Refinement Center 4: 155.0 unit

In [46]:
# proportion of canola oil is transshipped

# Calculate the total transshipped amount
total_transshipped4 = sum(y_4[i, k].X for i in range(15) for k in range(2) if y_4[i, k].X > 0)

# Calculate the total produced amount
total_produced4 = sum(x_4[i, j].X for i in range(25) for j in range(5) if x_4[i, j].X > 0) + total_transshipped4

# Calculate the proportion
proportion_transshipped4 = total_transshipped4 / total_produced4

print(f"Proportion of Canola Oil Transshipped: {proportion_transshipped4:.2%}")

Proportion of Canola Oil Transshipped: 30.00%
